# Homework 10

In [67]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS

In [68]:
Hitters = pd.read_csv('C:\\Users\\lorentsen\\Documents\\Python\\Math-180-Applied-Statistics-and-Machine-Learning\\ISLRv2_data\\ALL CSV FILES - 2nd Edition\\Hitters.csv')
Hitters.dropna(inplace=True)
y = Hitters['Salary']
x = Hitters.drop('Salary', axis=1)

## Problem 1
### part a)

In [82]:
for i in range(1, x.shape[1]):
    if x.iloc[:, i].dtype == 'object':
        continue
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()
# print(x)
Y = (y - y.mean()) / y.std()
x['NewLeague'] = x['NewLeague'].map({'A': 1, 'N': 0})
x['League'] = x['League'].map({'A': 1, 'N': 0})
x['Division'] = x['Division'].map({'E': 1, 'W': 0})


### part b)

In [ ]:
import statsmodels.api as sm

In [88]:
model = sm.OLS(Y, x).fit()
model.params

AtBat       -0.000081
Hits         0.215881
HmRun        0.075211
Runs        -0.099478
RBI         -0.102530
Walks        0.214055
Years        0.048549
CAtBat      -1.787843
CHits        1.091689
CHmRun      -0.025533
CRuns        0.836802
CRBI         0.596879
CWalks      -0.300279
League      -0.226753
Division     0.231789
PutOuts      0.157102
Assists      0.100293
Errors      -0.078106
NewLeague    0.073775
dtype: float64

### part c)

We have
\begin{equation}
f(X) = \beta_0 + \sum^K_{k=1} \beta_k g(w_{k0}+\sum^p_{j=1} w_{kj}X_j)
\end{equation}
But we have $K=0$ and we connect directly to the output layer. So we have
\begin{equation}
f(X) = \beta_0 + w_{10}+\sum^p_{j=1} w_{1j}X_j
\end{equation}
Where we can just define a new $\beta_0$ to be $\beta_0 + w_{10}$ and $\beta_j = w_{1j}$, so we get linear regression. \\

### part d)


In [ ]:
model = MS(Hitters.columns.drop('Salary'), intercept = False)
X = model.fit_transform(Hitters)
Y = Hitters['Salary']

X['NewLeague'] = X['NewLeague'].map({'A': 1, 'N': 0})
X['League'] = X['League'].map({'A': 1, 'N': 0})
X['Division'] = X['Division'].map({'E': 1, 'W': 0})

### part e)

In [ ]:
import keras
from keras import layers
from keras import ops

In [ ]:
modelnn = keras.Sequential(
    [
        # Input(shape),
        layers.Dense(1, input_shape=(X.shape[1],))
    ]
)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### part f)

In [ ]:
modelnn.compile(loss='mean_squared_error', optimizer='adam')
modelnn.fit(X, Y, epochs=200, verbose=0)

In [ ]:
showparams = pd.DataFrame()
showparams['glm'] = model.params 
showparams['nn'] = modelnn.get_weights()[0]
showparams.index = model.params.index
print(showparams)

                glm        nn
AtBat     -0.000081  0.077738
Hits       0.215881  0.950418
HmRun      0.075211 -0.005721
Runs      -0.099478  1.069494
RBI       -0.102530  0.981250
Walks      0.214055  0.181277
Years      0.048549  0.020214
CAtBat    -1.787843  0.182441
CHits      1.091689 -0.322822
CHmRun    -0.025533  0.335356
CRuns      0.836802 -0.333735
CRBI       0.596879  0.297414
CWalks    -0.300279 -0.057946
League    -0.226753 -0.010531
Division   0.231789  1.495157
PutOuts    0.157102  0.220071
Assists    0.100293 -0.018021
Errors    -0.078106  0.039753
NewLeague  0.073775 -0.293010
